## Reinforcement learning with Q-learning 

### Setting library

In [1]:
# import libraries
import numpy as np
import gym
import random
import time

from IPython.display import clear_output

### Step 1: Create the environment

In [2]:
# can query information about the environment, sample states and actions, 
# retreive rewards, and have agent navigate the envorinment
env = gym.make('FrozenLake-v0')

### Step 2: Create the Q-table and initialize it

In [3]:
# size of the action space 
action_space_size = env.action_space.n
# size of the state space in the environment
state_space_size = env.observation_space.n

In [4]:
# build Q-table and fill it with zeros
q_table = np.zeros((state_space_size, action_space_size))

In [5]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Step 3: Initializing Q-learning hyper-parameters

In [6]:
# create and initialize all the parameters needed to implement the Q-learning algorithm.
num_episodes = 10000 # number of plays during training
max_steps_per_episode = 100 # max number of steps agent allowed to take in a single episode

learning_rate = 0.1
discount_rate = 0.99

# related to the exploration-exploitation trade-off using epsilon-greedy policy
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01   # max and min are the bounds to how large or small our exploration rate can be
exploration_decay_rate = 0.001

# Note: exploration rate was represented by epsilon 

Note: The lower the exploration decay rate, the longer the agent will be able to explore. With 0.01 as the decay rate, the agent was only able to explore for a relatively short amount of time until it went into full exploitation mode without having a chance to fully explore and learn about the environment. Decreasing the decay rate to 0.001 allowed the agent to explore for longer and learn more.

If the exploration decay rate is too large, the 2nd term in the “exploration-rate update” is ≈ 0 (because the exponential term is ≈ 0). The impact is that subsequent epsilon-greedy searches get stuck in an “exploitation” mode, since the exploration rate converges to "min_exploration_rate" (little to no exploration occurs). 

These behavior would come much more clear if the game was deterministic (no slipping on ice), since the slippery situation adds a randomness which contributes to hide the phenomenon.

### Step 4: The Q learning algorithm

Source: https://www.kaggle.com/sandovaledwin/q-learning-algorithm-for-solving-frozenlake-game/code

In [7]:
rewards_all_episodes = []

In [8]:
# Step 4: Q-learing algorithm 

# everything that happends within a single episode
for episode in range(num_episodes):
    state = env.reset() # reset the state of the environment back to the starting state
    
    done = False # keeps track of whether or not the episode is finished so initialize to false for the starting
    rewards_current_episode = 0 # keep track of the current rewards within an episode set to zero at the begining of each episode
    
    # everything that happends within a single time step within each episode
    for step in range(max_steps_per_episode):
        
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0,1) # probability of exploring or exploiting the environment on this timne step
        #if exploration_rate_threshold > exploration_rate and ~np.all(q_table[state,:]==0):
        if exploration_rate_threshold > exploration_rate:
            # Exploit: Greedy Policy, only if the q values for the state are NOT all 0
            action = np.argmax(q_table[state,:])
        else:
            # Explore
            action = env.action_space.sample()
        
        # info --> diagnostic information regarding the environment
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) \
        + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
            
        # Exploraiton rate decay
        exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
        
        rewards_all_episodes.append(rewards_current_episode)                                 

In [9]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000

In [10]:
print("************* Average reward per thousand episodes *************\n")
for reward in rewards_per_thousand_episodes:
    print(count, "", str(sum(reward/1000)))
    count += 1000
    
# print update Q-table
print('\n\n********** Updated Q-table **********')
print(q_table)

************* Average reward per thousand episodes *************

1000  0.0
2000  0.0
3000  0.0
4000  0.0
5000  0.0
6000  0.0
7000  0.0
8000  0.0
9000  0.0
10000  0.0


********** Updated Q-table **********
[[0.54894128 0.49723681 0.51435069 0.50271982]
 [0.40771546 0.25452937 0.31320233 0.49460324]
 [0.40632777 0.41912727 0.41237965 0.45605739]
 [0.31664646 0.26525519 0.30803236 0.44215589]
 [0.56423568 0.26809739 0.27446237 0.26934401]
 [0.         0.         0.         0.        ]
 [0.17609613 0.15161969 0.36117057 0.11583256]
 [0.         0.         0.         0.        ]
 [0.4286229  0.41476959 0.43693872 0.59784548]
 [0.33796209 0.64692316 0.38619704 0.39963351]
 [0.69768942 0.43459488 0.31914848 0.38497603]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.49020933 0.59465529 0.76791042 0.60533449]
 [0.73100878 0.9158948  0.7565433  0.72039824]
 [0.         0.         0.         0.        ]]


The agent played 10,000 episodes, within each time step within an episode the agent received a reward of one if it reached the frisbee and otherwise it received a reward of zero. If the agen did indeed reach the frisbee then the episode finished at that time step. This means that for each episode the total reward received by the agent for the entire episode is either one or zero. For the first thousand episodes we can interpret the first score in the printout as meaning that 18% of the time the agent recieved a reward of one and won the episode. And by the last thousand episodes from a total of 10,000 the agent was winning 70% of the time and from the grid enviroment we can see that it is more likely for the agent to fall in a hole or perhaps reach the  the max number of time steps than it is to reach the frisbee. But reach the frisbee 70% of the time by the end of the training is not too bad especially since the agent had not explicit instructions to reach the frisbee in any case. It learned through reinforcement that this is the correct direction/action to do.

We observe from the print out that over time during training we can see the average rewards per thousand episodes did indeed progress over time. When the algorithm first started training for the first thousand episodes only averaged a rewards of 0.018 but the time it got to its last thousand episodes the reward drastically improved to 0.7. 

In [11]:
# Step 5: Use our Q-table to play FrozenLake !

# List of rewards
rewards = []

# 2 For life or until learning is stopped (num_episodes --> 3)
for episode in range(3):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    print('****** EPISODE ', episode+1, '*******\n\n\n')
    time.sleep(1) # allow print out to be read before disappearing
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True) # clear output from jupyter notebook cell
        env.render() # render current state of the environment to display where the agent is in the grid
        time.sleep(0.3) # allows one to see the current state of the environment before moving on to the next time step
        
         # Take the action (index) that have the maximum expected future reward given that state.
        # Set the action to the action with the highest Q-value in the Q-table for the current sate.
        action = np.argmax(q_table[state,:])
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action) # take action
        
        # If done (if we're dead) : finish episode
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print('***** You reached the goal! *****')
                time.sleep(3)
            else:
                print('**** You fell through a hole! *****')
                time.sleep(3)
                
            # clear output from jupyter cell
            clear_output(wait=True)
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
            
        # Our new state is state
        state = new_state
# close environment
env.close()

Number of steps 33
